In [3]:
!git clone https://dagshub.com/RZeusHerMun/my-first-repo.git

Cloning into 'my-first-repo'...


In [4]:
cd my-first-repo

C:\Users\el_he\RNAs\T4 - Dogs & Cats\my-first-repo


In [15]:
import os
import dagshub
import mlflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from mlflow import start_run, end_run, log_metric
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from getpass import getpass
from IPython.core.magics import history
from mlflow.tracking.fluent import log_param
from tensorflow.python.platform.tf_logging import log
from mlflow.keras import log_model

In [7]:
!pip3 install dagshub --upgrade

In [10]:
dagshub.init("my-first-repo", "RZeusHerMun", mlflow=True)

Repository initialized!

In [11]:
username = "RZeusHerMun"
reponame = "my-first-repo"
repowner = "RZeusHerMun"
eta = 0.001

In [16]:
os.environ['MLFLOW_TRACKING_USERNAME'] = username
os.environ['MLFLOW_TRACKING_PASSSWORD'] = getpass('Password de Dagshub')


Password de Dagshub········


In [17]:
mlflow.tensorflow.autolog()

In [18]:
model = Sequential()
model.add(Input(shape=(28,28)))
model.add(Flatten()) 
model.add(Dense(512, activation='sigmoid'))
model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))) #Regularizacion L1L2
model.add(Dropout(0.2)) #Fraccion de enlaces a eliminar
model.add(Dense(200)) #Capa lineal , transformacion lineal sin funcion de activacion
model.add(Activation('tanh')) #Se puede agregar despues la funcion de activacion
model.add(Dense(400, activation='selu', kernel_regularizer=regularizers.L1(0.01) )) #Regularizacion L1
model.add(Dense(200, activation='elu', kernel_regularizer=regularizers.L2(l2=1e-4)) ) #Regularizacion L2
model.add(Dense(50,activation='exponential'))
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 200)               102600    
                                                                 
 activation (Activation)     (None, 200)               0         
                                                                 
 dense_3 (Dense)             (None, 400)               8

In [19]:
earlystop = EarlyStopping(monitor='val_loss',mode='min',restore_best_weights=True, patience=10,verbose=1)
filepath = "primer_modelo"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [20]:
dataset=mnist.load_data()
(x_train, y_train), (x_test, y_test) = dataset

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255  
x_test /= 255

num_classes=10
y_trainc = keras.utils.to_categorical(y_train, num_classes)
y_testc = keras.utils.to_categorical(y_test, num_classes)

In [21]:
mlflow.set_experiment(experiment_name="Experiment 1")

2023/10/02 21:58:08 INFO mlflow.tracking.fluent: Experiment with name 'Experiment 1' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/b33ed6c8e47b4bff832bf04b4f986c63', creation_time=1696305488274, experiment_id='0', last_update_time=1696305488274, lifecycle_stage='active', name='Experiment 1', tags={}>

In [23]:
with mlflow.start_run(nested=True):
  mlflow.log_param("Tasa_aprendizaje", eta)

  model.compile(loss="categorical_crossentropy",optimizer=Adam(learning_rate=eta),metrics=['accuracy'])
  historial = model.fit(x_train, y_trainc,
                      batch_size=128,
                      epochs=5,
                      verbose=1,
                      validation_data=(x_test, y_testc),
                      callbacks=[earlystop, checkpoint])
  """mlflow.log_metric("loss", historial.history["loss"][-1])
  mlflow.log_metric("loss_validation", historial.history["val_loss"][-1])
  mlflow.log_metric("accuracy", historial.history["accuracy"][-1])
  mlflow.log_metric("accuracy_validation", historial.history["val_accuracy"][-1])"""
  loaded_model = tf.keras.models.load_model("primer_modelo")
  log_model(loaded_model, "mejores_modelos")
mlflow.end_run()

Epoch 1/5
467/469 [============================>.] - ETA: 0s - loss: 0.3380 - accuracy: 0.9689
Epoch 1: val_loss improved from 0.36065 to 0.31435, saving model to primer_modelo
INFO:tensorflow:Assets written to: primer_modelo\assets


INFO:tensorflow:Assets written to: primer_modelo\assets


469/469 [==============================] - 16s 30ms/step - loss: 0.3377 - accuracy: 0.9689 - val_loss: 0.3144 - val_accuracy: 0.9676
Epoch 2/5
467/469 [============================>.] - ETA: 0s - loss: 0.2809 - accuracy: 0.9740
Epoch 2: val_loss improved from 0.31435 to 0.28951, saving model to primer_modelo
INFO:tensorflow:Assets written to: primer_modelo\assets


INFO:tensorflow:Assets written to: primer_modelo\assets


469/469 [==============================] - 14s 29ms/step - loss: 0.2809 - accuracy: 0.9740 - val_loss: 0.2895 - val_accuracy: 0.9710
Epoch 3/5
468/469 [============================>.] - ETA: 0s - loss: 0.2541 - accuracy: 0.9792
Epoch 3: val_loss improved from 0.28951 to 0.25192, saving model to primer_modelo
INFO:tensorflow:Assets written to: primer_modelo\assets


...

INFO:tensorflow:Assets written to: primer_modelo\assets


1/1 [==============================] - 0s 90ms/step
INFO:tensorflow:Assets written to: C:\Users\el_he\AppData\Local\Temp\tmpgcw022cs\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\el_he\AppData\Local\Temp\tmpgcw022cs\model\data\model\assets
2023/10/02 22:09:07 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\el_he\AppData\Local\Temp\tmpjixlny1x\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\el_he\AppData\Local\Temp\tmpjixlny1x\model\data\model\assets
